# [CIFAR-10 Python](https://www.kaggle.com/datasets/pankrzysiu/cifar10-python) with Pre-trained model VGG16 and data Augmentation

Author: [1chooo](https://1chooo.com)

In [11]:
import keras
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras.optimizers import SGD, Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import (BatchNormalization, Conv2D, Dense,
                                     Dropout, Flatten, MaxPooling2D)

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

img_rows, img_cols, channels= 32, 32, 3

for i in range(0,9):
    plt.subplot(330 + 1 + i)
    plt.imshow(x_train[i])
plt.show()

Set up image augmentation

In [13]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

datagen.fit(x_train)

See example augmentation images

In [ ]:
for X_batch, y_batch in datagen.flow(x_train, y_train, batch_size=9):
    for i in range(0, 9):
        plt.subplot(330 + 1 + i)
        plt.imshow(X_batch[i].astype(np.uint8))
    plt.show()
    break

Reshape into images

In [ ]:
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, channels)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, channels)
input_shape = (img_rows, img_cols, 1)
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

Convert integers to float; normalise and center the mean


In [16]:
x_train = x_train.astype("float32")  
x_test = x_test.astype("float32")
mean = np.mean(x_train)
std = np.std(x_train)
x_test = (x_test - mean) / std
x_train = (x_train - mean) / std

labels

In [17]:
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [21]:
def plot_training_history(history) -> None:
    _, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10), facecolor='w')  # Create 2 subplots

    # First plot for accuracy
    ax1.plot(history.history["accuracy"], label="train accuracy")
    ax1.plot(history.history["val_accuracy"], label="test accuracy")
    ax1.set_title("Model Accuracy")
    ax1.set_ylabel("Accuracy")
    ax1.set_xlabel("Epoch")
    ax1.legend(loc="upper left")

    # Second plot for loss
    ax2.plot(history.history["loss"], label="train loss")
    ax2.plot(history.history["val_loss"], label="test loss")
    ax2.set_title("Model Loss")
    ax2.set_ylabel("Loss")
    ax2.set_xlabel("Epoch")
    ax2.legend(loc="upper left")

    # Adjust layout for better spacing
    plt.tight_layout()
    plt.show()


In [22]:
# build and compile the model  (roughly following the VGG paper)

reg = None
num_filters = 32
ac = 'relu'
adm = Adam(
    learning_rate=0.001,
    decay=0, 
    beta_1=0.9, 
    beta_2=0.999, 
    epsilon=1e-08
)
opt = adm
drop_dense = 0.5
drop_conv = 0

model = keras.Sequential()

model.add(Conv2D(num_filters, (3, 3), activation=ac, kernel_regularizer=reg, input_shape=(img_rows, img_cols, channels),padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(num_filters, (3, 3), activation=ac,kernel_regularizer=reg,padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))   # reduces to 16x16x3xnum_filters
model.add(Dropout(drop_conv))

model.add(Conv2D(2*num_filters, (3, 3), activation=ac,kernel_regularizer=reg,padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(2*num_filters, (3, 3), activation=ac,kernel_regularizer=reg,padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))   # reduces to 8x8x3x(2*num_filters)
model.add(Dropout(drop_conv))

model.add(Conv2D(4*num_filters, (3, 3), activation=ac,kernel_regularizer=reg,padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(4*num_filters, (3, 3), activation=ac,kernel_regularizer=reg,padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))   # reduces to 4x4x3x(4*num_filters)
model.add(Dropout(drop_conv))

model.add(Flatten())
model.add(Dense(512, activation=ac,kernel_regularizer=reg))
model.add(BatchNormalization())
model.add(Dropout(drop_dense))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer=opt)

training without augmentation

In [ ]:
history=model.fit(
    x_train, 
    y_train, 
    batch_size=128, 
    epochs=100, 
    validation_data=(x_test, y_test)
)

training accuracy without dropout

In [ ]:
train_acc = model.evaluate(x_train,y_train,batch_size=128)

In [ ]:
plot_training_history(history)

Evaluate the model

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

In [ ]:
print('Test accuracy:', test_acc)

## Reference

[Image-Augmentation-in-Keras-CIFAR-10-](https://github.com/moritzhambach/Image-Augmentation-in-Keras-CIFAR-10-/blob/master/CNN%20with%20Image%20Augmentation%20(CIFAR10).ipynb)
